# Dependencies

In [11]:
from langchain.document_loaders import PyPDFDirectoryLoader, TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter 
from openai import OpenAI
from langchain_openai import ChatOpenAI
from langchain.chains import LLMChain
from pinecone import Pinecone, ServerlessSpec
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate 
import pandas as pd
import json
from dotenv import load_dotenv
load_dotenv()
import os
import time

In [5]:
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

# Loading the JSON file 
Now, each conversation is stored as a chat history json, where each key is the name of the user and the value is the message sent by user, and this flow is maintained in the conversation. Now, our aim is to load these into the format we want to store as a vector. For now, let us store the conversation as a list of jsons, where each json is a chat history of a conversation.

In [30]:
conversations = [
    {
    "Dalvi OP": "i am currently doing the Excalidraw Project i was watching the second video i created all the files and followed all the steps but when i am executing the code by pnpm run dev i am getting this error i am stuck in this error for 3 hours if someone can please help me solve this error it would be very helpful for me",
    "TA Tarun": "what is the error?",
    "Dalvi OP (later)": "??",
    "TA Tarun (later)": "are you using esbuild?",
    "Dalvi OP (response)": "I don't think so where should I check if I am using it or no",
    "TA Harry Potter": "You need to configure it inside the http-backend app",
    "TA Tarun (later)": "package.json of http-server",
    "TA Sumana": "use latest version of node",
    "Dalvi OP (response final)": "@TA Sumana @TA Tarun @TA Harry Potter thankyou all of you the issue is resolved :perfect:"
  },
  {
    "Samim Rezvi OP": "how does a jwt work...like it takes a string and converts it into something random...what is the role of key here? @TA Harry Potter @TA Sumana @TA SerBarristan",
    "TA Harry Potter": "google kiya?",
    "Samim Rezvi OP (response)": "yes key ka role samajh nhi aya",
    "TA Harry Potter (clarification)": "it's that, allows verification that the token wasn't tampered with.",
    "Samim Rezvi OP (clarification request)": "mtlb key encrypt and decrypt krta hai na?",
    "TA Harry Potter (clarification final)": "Nahi, JWT ke case mein key encrypt aur decrypt nahi karti, balki sign aur verify karne ke liye use hoti hai.",
    "akshiiittt": "Basically jwt authentication mein kaam aati hai, like vo ek random kuch text generate karta hai jisko ham sign ki madad se create karte hai jismein ham 3 values pass on krte, first one is the payload second is the secret key(jo sabse main hai jiski help se hi server verfiy kar pata hai ki ki jwt jo genrate hua hai voh sahi hai ya galat and third pe expiry time of that token). This is basically the main thing of the jwt and second verify hota hai jo client se send hota hai and server compare karta hai secret ki help se yeh sahi token hai ke nahi and then hi allow karta hai client ko aggey kuch bhi activities karne ke liye.",
    "Samim Rezvi OP (final)": "accha...samajh gya.... thanks man"
  },
  {
    "Shubha Sarkar OP": "I am struggling a lot when building the second brain app using Recoil. Fetch all data when the /dashboard component loads from backend show in UI. There are 3 endpoints in backend \"/create\" \"/edit:id\" \"/delete:id\". The UI should update immediately and also update in backend when user performs these operations.",
    "TA Harry Potter": "Screen recording of the issue?",
    "Shubha Sarkar OP (response)": "https://drive.google.com/drive/folders/1e1n6mXUls2DD6yxnPANFfnEMKiJgeznd?usp=sharing",
    "TA Harry Potter (later)": "I'll check. Outside my home rn",
    "TA Harry Potter (recommendation)": "You should use an atom for state storage and combine it with a useEffect to fetch initial data on /dashboard load. It ensures updates reflect instantly by modifying the atom after every operation."
  },
  {
    "TANAY": "I am creating a project in NextJs. It was running fine but suddenly an error occurred while trying to animate the dropdown menu. Then I don't know why hydration error occurred. An error in global.css happened. When I undid the changes and re-ran the server, the error was gone but my styles were also gone.",
    "TA Jack Sparrow": "Restart the server and try again. Also inspect the styles like if Tailwind is even working or not.",
    "TANAY (response)": "It is showing but not applied.",
    "TA Jack Sparrow (clarification)": "Check the style in inspect elements.",
    "TANAY (clarification request)": "Also now it is showing hydration error.",
    "TA Jack Sparrow (suggestion)": "Check Tailwind docs to set it up again."
  },
  {
    "Aniket": "Auth.tsx\nimport { useRef } from \"react\";\nimport { Button } from \"../components/Button\";\nimport { Input } from \"../components/Input\";\nimport { BACKEND_URL } from \"../../config\";\nimport axios from \"axios\";\n\nexport const Signup = () => {\n const usernameRef = useRef<HTMLInputElement>();\n const passwordRef = useRef<HTMLInputElement>();\n\n const handleSignup = async () => {\n try {\n const username = usernameRef.current?.value;\n // console.log(username)\n const password = passwordRef.current?.value;\n // console.log(password)\n\n await axios.post(BACKEND_URL + \"/api/v1/signup\", {\n username,\n password\n });\n } catch (error) {\n console.log(error);\n }\n alert(\"User signed up!\");\n };\n\n return (\n <div className=\"flex justify-center items-center h-screen w-screen bg-gray-200\">\n <div className=\"bg-white rounded-md min-w-48 flex items-center justify-center flex-col p-8\">\n <Input reference={usernameRef} type=\"text\" placeholder=\"Username\" />\n <Input reference={passwordRef} type=\"password\" placeholder=\"Password\" />\n <div className=\"w-full mt-4\">\n <Button\n variant=\"primary\"\n size=\"md\"\n text=\"Signup\"\n fullWidth={true}\n onClick={handleSignup}\n />\n </div>\n </div>\n </div>\n );\n};\n\nexport const Signin = () => {\n return (\n <div className=\"flex justify-center items-center h-screen w-screen bg-gray-200\">\n <div className=\"bg-white rounded-md min-w-48 flex items-center justify-center flex-col p-8\">\n <Input type=\"text\" placeholder=\"Username\" />\n <Input type=\"password\" placeholder=\"Password\" />\n <div className=\"w-full mt-4\">\n <Button variant=\"primary\" size=\"md\" text=\"Signin\" fullWidth={true} />\n </div>\n </div>\n </div>\n );\n};",
    "Aniket (mention)": "@TA (WebDev and Devops)",
    "TA Harry Potter": "did you test the api in postman?",
    "TA Tarun": "add origin: * inside cors()",
    "Aniket (response)": "Yes",
    "Aniket (try)": "Ok let me try",
    "TA Harry Potter (advice)": "then enable it for the FE origin\n\napp.use(cors({\n origin: process.env.FRONTEND_URL || \"http://localhost:5173\",\n methods: ['GET', 'POST', 'PUT', 'DELETE'],\n credentials: true\n}));\nif you wanna enable for all origin",
    "TA Harry Potter (repeat)": "tarun add origin: ",
    "Aniket": "Updated the code still doesn't work",
    "TA Harry Potter": "app.use(cors(\"\")) this?",
    "Aniket": "you mean app.use(cors({\n origin: \"*\",\n})) ?",
    "TA Harry Potter (confirmation)": "yeah",
    "Aniket": "No",
    "TA Harry Potter (question)": "did you rebuild the server?",
    "Aniket": "Yes I did",
    "TA Harry Potter (suggestion)": "try logging the inputs in server routes and see if you are getting right or not",
    "Crytek": "reinstall the cors and re-import the cors module",
    "Aniket (resolved)": "The issue is resolved now 🎉. I noticed that whenever I add \"return\" statement it gave me this error. When I removed the return statement, it worked. Can anyone explain this why"
  },

]

# OpenAI Assistant Inferencing
Now, we have designed an assistant in openAI which will assist to summarise these chats and get them as valid json Q&A pairs. Below, we can see the process of inferencing the assistant and fetching the output.

In [49]:
openAI_client = OpenAI(api_key=OPENAI_API_KEY)
assistant_id = openAI_client.beta.assistants.list().data[0].id

Now, we have to create a thread. A thread here represents a conversation between a user and an assistant. Along with that, we should also sequentially add the messages from the conversation list to the thread.

In [86]:
thread = openAI_client.beta.threads.create()

def create_QAPair(conversation):
    openAI_client.beta.threads.messages.create(
        thread_id=thread.id,
        role="user",
        content= [{
            "type" : "text",
            "text" : json.dumps(conversation)
        }]
    )
            
    run = openAI_client.beta.threads.runs.create(
        thread_id=thread.id,
        assistant_id=assistant_id,
    )
    
    return run

def check_status(thread, run):
    while True:
        run = openAI_client.beta.threads.runs.retrieve(
            thread_id=thread.id,
            run_id=run.id
        )
        if run.status == 'completed':
            messages = openAI_client.beta.threads.messages.list(
                thread_id=thread.id
            )
            return messages
        elif run.status in ['failed', 'cancelled', 'expired']:
            raise Exception(f"Run ended with status: {run.status}")
        time.sleep(1)
         

Now, let us run the thread individually for each conversation, get the output and append it to the list of outputs.

In [102]:
def get_QAs(conversations):
    QAs = []
    for conversation in conversations : 
        run = create_QAPair(conversation)
        messages = check_status(thread, run)
        if(messages):
            QAs.append(messages.data[0].content[0].text.value)
        else:
            QAs.append({"question" : "Not available", "answer" : "Not available"})
    return QAs

In [103]:
QAs = get_QAs(conversations)

In [104]:
QAs

['{"question":"I am getting an error while executing the code with \'pnpm run dev\' during the Excalidraw Project. Can someone help me solve this error?","answer":"The issue was resolved by checking the configuration inside the http-backend app and ensuring the use of the latest version of Node.js. The teaching assistants suggested these steps, which helped in resolving the error."}',
 '{"question":"How does a JWT work, and what is the role of the key in it?","answer":"A JWT (JSON Web Token) works by taking a string and converting it into a token that can be used for authentication. The key plays a crucial role in signing and verifying the token, ensuring that it hasn\'t been tampered with. The JWT consists of three parts: the payload, the secret key, and the expiry time. The payload contains the claims, the secret key is used by the server to verify the authenticity of the token, and the expiry time defines how long the token is valid. The server uses the secret key to compare and ver

Now, each chunk will ideally belong to a documnent stored in S3 bucket in AWS. So, for now we will assume that the **KEY** of that document is in the format as given below which we will be using in the metadata. To simiplify the process, each entry in the vectorDB should have : 

* **ID** : The unique ID of the document, which will be a combination of the document key and the chunk number.
* **VALUES** : The embedding of the chunk, as generated by the OpenAI text-embedding-3-small model.
* **METADATA** : The metadata of the document, which will include the document key and the chunk number.
    *  **USER_ID** : The ID of the user in our system.
    *  **DOCUMENT_TYPE** : The type of the document from either "pdf" or "text".
    *  **KEY** : The key of the document in the S3 bucket, which defines the location of the document.
    *  **CHUNK** : The text of the chunk.
    *  **PAGE_NUMBER** : The page number of the chunk in the document.
    *  **CHUNK_INDEX** : The number of the chunk in the document.

Since most of the information will be given by the backend server, for now we will use dummy values for the metadata.

In [19]:
KEY = "61f100abx/pdf/W9-react"
USER_ID = "61f100abx"
DOCUMENT_TYPE = "pdf"

Let us now create the function to create vectors in our desired format as defined above.

In [20]:
def create_vectors(text_chunks,KEY,USER_ID,DOCUMENT_TYPE):
    v = []
    chunk_num = 0
    
    for chunk in text_chunks: 
        page_num = chunk.metadata["page"]
        
        entry = {}
        entry["id"] = f"{KEY}_PAGE_{page_num}_CHUNK_{chunk_num}"
        entry["values"] = get_embedding(chunk.page_content)
        entry["metadata"] = {
            "userID" : USER_ID,
            "type" : DOCUMENT_TYPE,
            "key" : KEY,
            "chunk" : chunk.page_content,
            "page_number" : chunk.metadata["page"],
            "chunk_number" : chunk_num
        }
        
        chunk_num += 1
        v.append(entry)
        
    return v

In [21]:
vectors = create_vectors(text_chunks,KEY,USER_ID,DOCUMENT_TYPE)

With this, we have our vectors stored in the ideal format to be pushed into the vector DB. Let us now push the vectors into the vectorDB of pinecone.

# Pushing the Vectors into the Pinecone Index

In [22]:
index_name = "documents"
while not pc.describe_index(index_name).status['ready']:
    time.sleep(1)

index = pc.Index(index_name)

index.upsert(
    vectors=vectors
)

{'upserted_count': 108}

# Querying the Vectors
We shall now query the vectors to check if the vectors have been stored correctly in the pinecone index, and how does this exactly work. We will fetch the relevant vectors from the pinecone index. For that, we will create a function which takes a text query, converts into to an embedding and queries the pinecone index to get the most similar texts from the vectors stored in the index.

In [25]:
def get_relevant_chunks(query,userID,KEY):
    query_vector = get_embedding(query)
    
    results = index.query(
        vector = query_vector,
        top_k = 10,
        include_values = False,
        include_metadata = True,
        filter={
            "userID" : userID,
            "key" : KEY
        }
    )
    
    relevant_texts = []
    for record in results['matches']:
        text = {}
        text['score'] = record['score']
        text['text'] = record['metadata']['chunk']
        text["reference"] = int(record["metadata"]["page_number"]) + 1
        relevant_texts.append(text)
    
    return relevant_texts

Finally, we can create a QA system which will take a query and return the most relevant chunks from the PDF document.

In [26]:
import sys 
while True:
    user_input = input(f"Input Prompt: ")
    if user_input=='exit':
        print( 'Exiting')
        sys.exit()
    if user_input == '':
        continue
    
    docs = get_relevant_chunks(user_input,USER_ID,KEY)
        
    for doc in docs:
        print(f"Rank {doc['score']} \n Reference {doc['reference']} \n Answer: \n {doc['text']}")
        print("------------------------")

    print("------------------------------------------------------------------------------------------------------------------------")
        

Rank 0.597881198 
 Reference 6 
 Answer: 
 The useEffect hook let s you perform side ef fects in functional component s in a 
safe, predictable way:
useEffect (() => {
  // Code here is the "effect" — this is where side effects  
happen
  fetchData ();
  // Optionally, return a cleanup function that runs when t
he component unmounts.
  return () => {
    // Cleanup code, e.g., unsubscribing from an event or c
------------------------
Rank 0.555085778 
 Reference 7 
 Answer: 
 Optional Cle anup If your side ef fect needs cle anup (e.g., unsubscr ibing 
from a WebSocket, clearing intervals), useEffect allows you to return a 
function that R eact will c all when the component unmount s or before re-
running the ef fect.
T o  r e c a p
useEffect is a Hook that let s you perform side ef fects in functional component s. 
It can be used f or data fetching, subscr iptions, or manuall y changing the DOM.
L i n k e d i n  l i k e  t o p b a r
------------------------
Rank 0.488263339 
 Referenc

SystemExit: 

With this, our pipeline is complete and we can now move on to the next steps which is sending these relvant documents to the LLM to answer our query.

# Prompt Template for the LLM
Here, we will need to define the prompt for the LLM to answer the query. The LLM will be given the query and the relevant documents, and it will be expected to return the answer to the query. 

In [27]:
query_prompt_template = """
    You are a specialised AI document analyser working at an edtech startup, and you will be assisting the users to answer their queries. You will be given 
    the top relevant documents and you have to use those to answer the query asked by the user, which will be given to you below. 
    In the relevant documents,you will be given the cosine similarity score, the reference (which is the page number where this 
    text was in the document) and the text itself. You can in you answer integrate the reference to build authenticity of your answer, 
    by precisely writing it like (reference page : page_num)
    
    \n\n User Query : {query}
    \n\n Documents : {documents}
    
    MAKE SURE YOU DO NOT ANSWER FROM ANYTHING APART FROM THE DOCUMENTS GIVEN TO YOU. 
"""

In [28]:
query_prompt = PromptTemplate(
    input_variables=["query","documents"],
    template=query_prompt_template
)

# Initializing the LLM Client and Chain for RAG Model

In [29]:
chat = ChatOpenAI(
    temperature = 0,  
    model = "gpt-4o",
    openai_api_key = OPENAI_API_KEY
)

In [30]:
query_chain = LLMChain(
    llm=chat,
    prompt=query_prompt
)

# Q&A System using the chain

In [32]:
user_query = "could you explain useState with some examples?"
docs = get_relevant_chunks(user_query,USER_ID,KEY)

# Run the chain
response = query_chain.invoke({
    "query": user_query,
    "documents": docs
})

# Print the response
print("Response from LLM:")
print(response['text'])

Response from LLM:
The `useState` hook is a fundamental part of React that allows you to add state to functional components. It returns an array with two elements: the current state value and a function to update that state. Here's a basic example to illustrate how `useState` works:

1. **Counter Example**:
   ```javascript
   import React, { useState } from 'react';

   const Counter = () => {
     const [count, setCount] = useState(0);

     return (
       <div>
         <p>You clicked {count} times</p>
         <button onClick={() => setCount(count + 1)}>
           Click me
         </button>
       </div>
     );
   };
   ```
   In this example, `useState(0)` initializes the state variable `count` to 0. The `setCount` function is used to update the state whenever the button is clicked, incrementing the count by 1 (reference page: 1).

2. **Posts Example**:
   ```javascript
   import { useState } from "react";
   import { PostComponent } from "./Post";

   function App() {
     co

Here, we have successfully built the RAG model and the Q&A system using the chain. With this, we get the functionality to query the relevant documents and get the answer to the query.

# Deleting via Pinecone (Listing indexes with prefix and then deleting the index)

In [ ]:
index_name = "documents"


index = pc.Index(index_name)
list1 = []
for ids in index.list(prefix=KEY):
  list1.append(ids)

In [16]:
KEY = "674313bc5cbbf2d8da2a4649/pdf/"
res = index.delete([ids for ids in index.list(prefix=KEY)])
res

{}